In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from skimage import io, transform
from skimage.color import rgb2lab, lab2rgb

# Load the image and resize it
image = io.imread('assets/pinnacle.png')
# Chop out the alpha channel
image = image[:,:,:3]
image = transform.resize(image, (512, 512))


In [ ]:
plt.imshow(image)

In [ ]:

voronoi_points = np.random.randint(0, 4096, (6096, 2))

# Create Voronoi diagram
vor = Voronoi(voronoi_points)

voronoi_map = np.zeros((4096, 4096, 3), dtype=np.int8)

# Project the colors from the image to the Voronoi map
for region in vor.regions:
    for index in region:
        if index != -1:  # Ignore the points at infinity
            x, y = vor.vertices[index]
            if (0 < x < 4096 and 0 < y < 4096):
                x, y = int(x), int(y)
                # Get the color from the closest pixel in the image
                i, j = np.round([y/16, x/16]).astype(int)
                voronoi_map[y, x, :] = 255


print(voronoi_map.shape, voronoi_map.dtype)
plt.imshow(voronoi_map)

In [ ]:

voronoi_points = np.random.randint(0, 4096, (6096, 2))

# Create Voronoi diagram
vor = Voronoi(voronoi_points)

voronoi_map = np.zeros((4096, 4096, 3), dtype=np.float16)

# Project the colors from the image to the Voronoi map
for region in vor.regions:
    for index in region:
        if index != -1:  # Ignore the points at infinity
            x, y = vor.vertices[index]
            # Get the color from the closest pixel in the image
            i, j = np.round([y/8, x/8]).astype(int)
            voronoi_map[int(y), int(x)] = image[i, j]

# Convert colors to LAB for better comparison
voronoi_map_lab = rgb2lab(voronoi_map)
image_lab = rgb2lab(image)

# Flood fill
change_occurred = True
while change_occurred:
    change_occurred = False
    for (p1, p2) in vor.ridge_points:
        r1 = vor.point_region[p1]
        r2 = vor.point_region[p2]
        
        # Check if one region is empty and the other is not
        if np.all(voronoi_map_lab[r1] == 0) and not np.all(voronoi_map_lab[r2] == 0):
            voronoi_map_lab[r1] = voronoi_map_lab[r2]
            change_occurred = True
        elif np.all(voronoi_map_lab[r2] == 0) and not np.all(voronoi_map_lab[r1] == 0):
            voronoi_map_lab[r2] = voronoi_map_lab[r1]
            change_occurred = True

# Convert back to RGB for visualization
voronoi_map = lab2rgb(voronoi_map_lab)

# Show the result
plt.imshow(voronoi_map)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d
from PIL import Image

# Load the image and resize it to 4096x4096
img = Image.open('assets/pinnacle.png')
img = img.resize((4096, 4096))
img_array = np.array(img)

# Define a random set of points for the Voronoi diagram
points = np.random.randint(4096, size=(6096, 2))

# Create the Voronoi object
vor = Voronoi(points)

# Plot
fig, ax = plt.subplots(figsize=(20, 20))
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='black')

# Assign colors from the image to the Voronoi regions
for region in vor.regions:
    if not -1 in region:  # Exclude regions with vertices outside the diagram
        polygon = [vor.vertices[i] for i in region]
        # Use the color from the center of the region in the original image
        region_center = points[vor.point_region[region[0]]]
        color = img_array[region_center[1], region_center[0]] / 255  # Convert to [0, 1] range
        ax.fill(*zip(*polygon), color=color)

# Set the figure size
plt.xlim(0, 4096)
plt.ylim(0, 4096)

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def load_image():
    # Load the image and resize it to 4096x4096
    img = Image.open('assets/pinnacle.png')
    img = img.resize((4096, 4096))
    img_array = np.array(img)
    return img_array

from scipy.spatial import Voronoi, voronoi_plot_2d

# Define a random set of points for the Voronoi diagram
points = np.random.randint(4096, size=(12192, 2))

# Create the Voronoi object
vor = Voronoi(points)

# Plot
fig, ax = plt.subplots(figsize=(10, 10))
voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='black')

# Assign colors from the image to the Voronoi regions
for idx, region in enumerate(vor.regions):
    if not -1 in region:  # Exclude regions with vertices outside the diagram
        polygon = [vor.vertices[i] for i in region]
        # Use the color from the center of the region in the original image
        if vor.point_region[idx] < len(points):  # Ensure the region corresponds to a point
            region_center = points[vor.point_region[idx]]
            color = img_array[region_center[0], region_center[1]] / 255  # Convert to [0, 1] range
            #ax.fill(*zip(*polygon), color=color, alpha=.5)

# Set the figure size
plt.xlim(0, 4096)
plt.ylim(0, 4096)

# Overlay the original image with alpha 0.4
ax.imshow(img_array, alpha=0.9)

# Show the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d, ConvexHull


rng = np.random.default_rng()
points = rng.random((10,2))
points = np.append(points, [[999,.5], [-999,.5], [.5,-999], [.5,999]], axis = 0)

# Assign a random color to each point
colors = rng.random((points.shape[0] + 1, 3))

vor = Voronoi(points)

#fig = voronoi_plot_2d(vor)

#fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange', line_width=2, line_alpha=0.6, point_size=2)

# Plot the points in blue circles
#plt.scatter(points[:,0], points[:,1], s=40, c=colors[:-1])

# Plot the centroids of the Voronoi polygons with the same color as the corresponding point
"""
for i, ridge_points in enumerate(vor.ridge_points):
    if -1 in ridge_points:
        continue
    polygon = vor.vertices[vor.ridge_vertices[i]]
    centroid = np.mean(polygon, axis=0)
    plt.scatter(ridge_points[0], ridge_points[1], s=50, c=colors[ridge_points[0]])
    #plt.scatter(centroid[0], centroid[1], s=50, c=colors[ridge_points[0]])
"""

"""
# Plot the centroids of the Voronoi polygons in red
for i, region in enumerate(vor.regions):
    polygon = vor.vertices[region]
    print(region, polygon)
    if len(polygon) == 0:
        continue
    centroid = np.mean(polygon, axis=0)
    #hull = ConvexHull(polygon)
    plt.scatter(centroid[0], centroid[1], s=50, c=colors[i])
    #plt.plot(polygon[hull.vertices,0], polygon[hull.vertices,1], 'k-', lw=2)
"""

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

#len(vor.ridge_points), len(vor.ridge_vertices)
#print(vor.vertices)
vertices = np.clip(vor.vertices, 0, 1)
pclip = np.clip(points, 0, 1)
ax.scatter(vertices[:, 0], vertices[:, 1])
ax.scatter(pclip[:,0], pclip[:,1])

for i, region in enumerate(vor.regions):
    if -1 in region:
        continue
    polygon = vertices[region]
    #print(region, polygon)
    if len(polygon) == 0:
        continue
    polygon = [vertices[i] for i in region]
    polygon = np.clip(polygon, 0, 1)
    #print(polygon)
    ax.fill(*zip(*polygon), alpha=.5, color=colors[i])
    #centroid = np.mean(polygon, axis=0)
    #hull = ConvexHull(polygon)
    #plt.scatter(centroid[0], centroid[1], s=50, c=colors[i])
    #plt.plot(polygon[hull.vertices,0], polygon[hull.vertices,1], 'k-', lw=2)